In [ ]:
import os
import cv2 as cv
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from glob import glob
from datetime import datetime

In [ ]:
DATA_PATH = r'/scratch/gallowaa/cciw/Data'

In [ ]:
# Search for all video files on Google Drive...
all_images = glob(os.path.join(DATA_PATH,'Videos_and_stills/GLNI/*/*/*/Images/Quad*/*.jpg'))
#all_images = glob(os.path.join(DATA_PATH,'Videos_and_stills/GLNI/*/*/May.*/Stills/Quad*/*.jpg'))

In [ ]:
imagetable_path = os.path.join(DATA_PATH, 'Tables', 'ImageTable.csv')
image_df = pd.read_csv(imagetable_path, index_col=0)

In [ ]:
analysis_path = os.path.join(DATA_PATH, 'Tables', 'Analysis.csv')
dive_path = os.path.join(DATA_PATH, 'Tables', 'Dives.csv')

analysis_df = pd.read_csv(analysis_path, index_col=0, dtype={'Count':float})
dive_df = pd.read_csv(dive_path, index_col=0, parse_dates=['Date'])
data_df = pd.merge(analysis_df, dive_df, on='Dive Index', how='outer')
data_df.columns

In [ ]:
"""note: these indices were determined by visual 
inspection of the images for i) good illumination/
clarity, ii) approximately constant camera distance,
and iii) low cladophora cover.""" 
indices = np.concatenate((np.arange(50, 60), 
                          np.arange(78, 82),
                          np.arange(92, 94),
                          np.arange(103, 105),
                          np.arange(129, 133),
                          np.arange(156, 171),
                          np.arange(129, 133),
                          np.arange(196, 207),
                          np.arange(218, 222),
                          np.arange(236, 238),
                         ))
print(len(indices))

In [ ]:
dim = []
for i in indices:
    im = cv.imread(all_images[i])
    dim.append((i, im.shape[0]))
dim = np.asarray(dim)

In [ ]:
working_set = dim[dim[:, 1] == 7378][:, 0]

In [ ]:
%matplotlib

In [ ]:
#for i in range(280, 500):
i=350
im = cv.imread(all_images[i])
rgb = cv.cvtColor(im, cv.COLOR_BGR2RGB)
plt.imshow(rgb[::4, ::4, :])
plt.title('frame ' + str(i))
plt.show()
#plt.pause(0.1)

In [ ]:
root_fname = all_images[i].split('/')[-1].split('.')[0]
guid = image_df[image_df['Name'].str.contains(root_fname)]['Analysis Index'].astype('int64')
data_df[data_df['Analysis Index'].values == guid.values]

In [ ]:
plt.close('all')

In [ ]:
all_images[350]

In [ ]:
data_df[data_df['Dive Index'].values == 520]

In [ ]:
# Number of dives for each year-month
dive_df['Year-Month'] = dive_df['Date'].dt.strftime('%Y-%m')
dive_df_by_year_month = dive_df.groupby(by='Year-Month').count()['PSN']
dive_df_by_year_month.index = [datetime.strptime(year_month, '%Y-%m') for year_month in dive_df_by_year_month.index]
#dive_df_by_year_month

In [ ]:
YM = '2018-08'

save_path = os.path.join('/scratch/ssd/gallowaa/cciw/dataset_raw_v0-2-x', YM + '/land') # '/land or /port'

print(save_path)

dive_idx = dive_df[dive_df['Year-Month'].values == YM].index #['Dive Index']
print(dive_idx)

In [ ]:
# analysis indices for data collected in 2018-07
indices = data_df.loc[data_df['Dive Index'].isin(dive_idx)]['Analysis Index'].values

In [ ]:
#image_df[image_df['Analysis Index'].isin(indices)]
fnames = image_df[image_df['Analysis Index'].isin(indices)]['Name']

In [ ]:
data_df.loc[data_df['Dive Index'].isin(dive_idx)]

In [ ]:
#data_df[data_df['Analysis Index'] == 57]

In [ ]:
root = os.path.join(DATA_PATH,'Videos_and_stills/GLNI/')

In [ ]:
#fnames.values

In [ ]:
dim = []
month = 'Aug.' # Jul, Jun
#for i, f in enumerate(fnames.values[portrait_mode]):
for i, f in enumerate(fnames.values):
    tokens = f.split('.')[0].split('_')
    PSN = tokens[1].split('-')[0]
    QUAD = tokens[1].split('-')[1]
    file = root + PSN + '/2018/' + month + tokens[2].split('-')[-1] + '/Images/Quad' + QUAD + '/' + f.split('.')[0] + '.jpg'
    im = cv.imread(file)
    #rgb = cv.cvtColor(im, cv.COLOR_BGR2RGB)
    #plt.imshow(rgb[::4, ::4, :])
    #plt.title('frame ' + str(i))
    #plt.pause(0.1)
    dim.append((i, im.shape[0]))
dim = np.asarray(dim)

In [ ]:
vals, cts = np.unique(dim[:, 1], return_counts=True)
print(vals)
print(cts)
print(cts.sum())

port_mode = dim[:, 0][dim[:, 1] == 7378]
land_mode = dim[:, 0][dim[:, 1] == 4924]

In [ ]:
#port_mode

In [ ]:
#land_mode

In [ ]:
#root_fname

In [ ]:
#fnames.values[land_mode]

In [ ]:
#i = 12
month = 'Aug.' # Jul, Jun
#for i, f in enumerate(fnames.values[land_mode]):
#f = fnames.values[land_mode][i]

f = 'GLNI_3801-2_2018-08-13_image-2.jpg'
tokens = f.split('.')[0].split('_')
PSN = tokens[1].split('-')[0]
QUAD = tokens[1].split('-')[1]
file = root + PSN + '/2018/' + month + tokens[2].split('-')[-1] + '/Images/Quad' + QUAD + '/' + f.split('.')[0] + '.jpg'
im = cv.imread(file)
rgb = cv.cvtColor(im, cv.COLOR_BGR2RGB)

#imc = im[y_start:y_end, x_start:x_end, :]
#scale_percent = 75 # percent of original size
#width = int(imc.shape[0] * scale_percent / 100)
#height = int(imc.shape[1] * scale_percent / 100)

# resize image
#imcr = cv.resize(imc, (width, height))
#cv.imwrite(os.path.join(save_path, f.split('.')[0] + '_crop.jpg'), imcr)
#plt.figure(figsize=(6, 12))
plt.imshow(im)
#plt.title('frame ' + str(i) + ' of ' + str(len(fnames.values[portrait_mode])))

In [ ]:
# for portrait mode (7378, 4924)
"""
x_start, x_end = 1050, 4050
y_start, y_end = 2100, 5100
"""
# for landscape mode (4924, 7378)
x_start, x_end = 2100, 5100
#x_start, x_end = 2200, 5200
#y_start, y_end = 800, 3800
y_start, y_end = 1000, 4000

print(x_end - x_start)
print(y_end - y_start)

imc = im[y_start:y_end, x_start:x_end, :]

plt.figure(figsize=(6, 12))
plt.imshow(imc)

In [ ]:
scale_percent = 75 # percent of original size
width = int(imc.shape[0] * scale_percent / 100)
height = int(imc.shape[1] * scale_percent / 100)
dim = (width, height)

# resize image
imcr = cv.resize(imc, dim)
cv.imwrite(os.path.join(save_path, f.split('.')[0] + '_crop.jpg'), imcr)

In [ ]:
#width, height
width / 250

In [ ]:
'''
wvz = 250 # 340, 283, 242, 212
n_plot = imcr.shape[0] // wvz
fig, axes = plt.subplots(n_plot, n_plot, figsize=(12, 12))

for row in range(imcr.shape[0] // wvz):
    for col in range(imcr.shape[0] // wvz):
        axes[row, col].imshow(imcr[row*wvz:(1+row)*wvz, col*wvz:(col+1)*wvz])
        axes[row, col].axis('off')
plt.tight_layout()        
'''
#plt.savefig(file + '_viz.png')

In [ ]:
#(66 + 37 + 31) * 9**2

# You left off here...
plot Biomass/Count versus number of pixels labeled as mussel

In [ ]:
#%matplotlib inline
#i = 2
#root_fname = fnames.values[portrait_mode][i].split('/')[-1].split('.')[0]
#guid = image_df[image_df['Name'].str.contains(root_fname)]['Analysis Index'].astype('int64')
#data_df[data_df['Analysis Index'].values == guid.values]

guid = image_df[image_df['Name'].str.contains('3801-1_2018-08')]['Analysis Index'].astype('int64')
data_df[data_df['Analysis Index'].values == np.unique(guid.values)]

In [ ]:
#fnames.values[land_mode]#[i].split('/')

In [ ]:
#20 * 50

# After Labeling
`./labelme2voc.py /scratch/ssd/gallowaa/cciw/dataset_raw_v0-2-x/2018-06 /scratch/ssd/gallowaa/cciw/VOCdevkit/VOC2012_v020 --labels labels.txt` 


In [ ]:
mask_path = '/scratch/ssd/gallowaa/cciw/VOCdevkit/VOC2012_v024/SegmentationClassPNG'

In [ ]:
png_files = os.listdir(mask_path)
print(len(png_files))

In [ ]:
#png_files

In [ ]:
# count the number of pixels assigned "Zebra" in image masks
pix_ct = []
for i in range(len(png_files)):
    lab = cv.imread(os.path.join(mask_path, png_files[i]))
    _, counts = np.unique(lab, return_counts=True)
    pix_ct.append(counts[1])
pix_ct = np.asarray(pix_ct) 
pix_ct = pix_ct / pix_ct.max()

In [ ]:
lab_targets = np.zeros((len(png_files), 2)) # 0 = biomass, 1 = count
for i in range(len(png_files)):
    root_fname = png_files[i].split('.')[0][0:-5] 
    guid = image_df[image_df['Name'].str.contains(root_fname)]['Analysis Index'].astype('int64')
    row = data_df[data_df['Analysis Index'].values == np.unique(guid.values)]
    lab_targets[i, 0] = row['Biomass'].values
    lab_targets[i, 1] = row['Count'].values

In [ ]:
A = np.vstack([pix_ct, np.ones(len(pix_ct))]).T

lab_targets[np.isnan(lab_targets)] = 0

m, c = np.linalg.lstsq(A, lab_targets[:, 0], rcond=None)[0]
x = np.linspace(0, 1)

In [ ]:
fig, (ax, ax2) = plt.subplots(1, 2, sharex=True, figsize=(8, 4))

BIOMASS = 0
COUNTS = 1
ax.scatter(pix_ct, lab_targets[:, BIOMASS], marker='o', s=40, facecolors='none', edgecolors='b')
#ax.scatter(pix_ct[19], lab_targets[:, BIOMASS][19], marker='o', s=100, facecolors='none', edgecolors='k')
ax.set_ylabel('Mussel Biomass', color='b')
ax.set_xlabel('Normalized Segmentation Mask Pixel Count')
#ax.set_xlim(0, 1)
ax.plot(x, m*x + c, 'b')
ax.grid()

ax3 = ax2.twinx()  # instantiate a second axes that shares the same x-axis
ax2.scatter(pix_ct, lab_targets[:, COUNTS], marker='+', c='r')
ax3.set_ylabel('Mussel Count', color='r')
ax2.grid()

fname = 'mussel_biomass_and_count_versus_zebra_pixel_count_outliers'
fig.savefig(fname + '.png', format='png')
fig.savefig(fname + '.eps', format='eps')

In [ ]:
idx = (lab_targets[:, BIOMASS] > 300) & (pix_ct < 0.15)#.argmax()

In [ ]:
idx.argmax()

In [ ]:
png_files[19]

In [ ]:
GLNI_3540-2_2018-07-4_image_1_crop.json